In [1]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

In [2]:
# stack = 'MD589'
# sec = 165

stack = 'MD594'
sec = 289

dm = DataManager(stack=stack, section=sec, segm_params_id='tSLIC200', load_mask=False)

dm._load_image(['rgb'])

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/PIL/Image.py:2224: DecompressionBombWarning: Image size (167180288 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


In [3]:
patch_size = 224
half_size = patch_size/2
stride = 56

h, w = dm.image_rgb.shape[:2]

ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                     indexing='xy')
    
sample_locations = np.c_[xs.flat, ys.flat]

In [4]:
ys.shape

(254, 203)

In [21]:
# mask = imread(dm.data_dir+'/%(stack)s_thumbnail_aligned_cropped_mask/%(stack)s_%(slice_str)s_thumbnail_aligned_cropped_mask.png' % \
#        {'stack': dm.stack, 'slice_str': dm.slice_str}) > 0
dm.load_thumbnail_mask()
mask = dm.thumbmail_mask.astype(np.bool)
# mask_lossless = rescale(mask, 32)

In [6]:
patches = []

for x, y in sample_locations:

    patch = dm.image_rgb[y-half_size:y+half_size, x-half_size:x+half_size]
    patches.append(patch)
        
print len(patches), 'patches'

51562 patches


In [7]:
output_dir = '/home/yuncong/CSHL_data_patches/'+ stack +'_patches/%04d'%sec
if not os.path.exists(output_dir):
    os.makedirs(output_dir)    

In [8]:
sample_locations_inMask = []
sample_indices_inMask = []

for i, p in enumerate(patches):
    x, y = sample_locations[i]
    if mask[y/32,x/32]:
        sample_locations_inMask.append((x,y))
        sample_indices_inMask.append(i)
        cv2.imwrite(output_dir + '/%08d.jpg'%i, img_as_ubyte(p[..., ::-1]))

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/util/dtype.py:111: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [9]:
np.save('/home/yuncong/CSHL_data_patches/'+stack+'_%04d_sample_locations_inMask.npy'%sec, 
        np.asarray(sample_locations_inMask))

In [10]:
output_dir2 = '/home/yuncong/CSHL_data_patches/'+ stack +'_patches/%04d_reduced'%sec
if not os.path.exists(output_dir2):
    os.makedirs(output_dir2)

In [22]:
# MD589_165
# box_x = 43
# box_y = 205
# box_w = 170
# box_h = 143
# xmin = box_x*32
# xmax = (box_x+box_w)*32
# ymin = box_y*32
# ymax = (box_y+box_h)*32

# MD594_289
box_x = 37
box_y = 122
box_w = 393
box_h = 219
xmin = box_x*32
xmax = (box_x+box_w)*32
ymin = box_y*32
ymax = (box_y+box_h)*32

In [24]:
reduced_sample_indices = []
for i, (c,r) in enumerate(sample_locations):
    if r > ymin and r < ymax and c > xmin and c < xmax:
        if mask[r/32,c/32]:
            reduced_sample_indices.append(i)
            
print len(reduced_sample_indices), 'patches'

23435 patches


In [25]:
from joblib import delayed, Parallel

In [ ]:
# _ = Parallel(n_jobs=16)(delayed(os.system)('ln -s %(f1)s %(f2)s'%{'f1': output_dir + '/%08d.jpg'%i,
#                                     'f2': output_dir2 + '/%08d.jpg'%i}) for i in reduced_sample_indices)

In [ ]:
# output_dir3 = '/home/yuncong/CSHL_data_patches/'+ stack +'_patches/%04d_reduced_cp'%sec
# if not os.path.exists(output_dir3):
#     os.makedirs(output_dir3)

In [27]:
import time
t = time.time()

_ = Parallel(n_jobs=16)(delayed(os.system)('cp %(f1)s %(f2)s'%{'f1': output_dir + '/%08d.jpg'%i,
                                            'f2': output_dir2 + '/%08d.jpg'%i}) for i in reduced_sample_indices)

print time.time() - t

172.332828045


In [28]:
print 'tar -I pigz -cf ' + stack + '_%04d_reduced.tar '%sec + '%04d_reduced'%sec

tar -I pigz -cf MD594_0289_reduced.tar 0289_reduced


In [29]:
np.save('/home/yuncong/CSHL_data_patches/'+stack+'_%04d_sample_locations_reduced.npy'%sec, 
        sample_locations[reduced_sample_indices])

In [ ]:
# for i in reduced_sample_indices:
#     os.system('ln -s %(f1)s %(f2)s'%{'f1': output_dir + '/%08d.jpg'%i,
#                                     'f2': output_dir2 + '/%08d.jpg'%i})